# Einarbeitung in Python-Chess
   - Eröffnungsbibliothek
   - Endspiel-Bibliothek: Syzygy
   - Version 0: zufällige Züge auswählen im Mittelspiel

WICHTIG: Alles soll als Jupyter-Notebook implementiert werden.  
Jede Funktion muss ausführlich dokumentiert werden (mittleres Markdown + LaTeX), d.h.
- a) Welche Argumente erhält die Funktion.
- b) Welches Ergebnis wird berechnet.
- c) Welche Seiteneffekte treten auf (falls Seiteneffekte auftreten).
- d) Bei komplizierter Logik soll auch Algorithmus beschrieben werden.

Testen:
- Abspeichern einer Partie als Datei auf Festplatte in algebraischer Notation.
- Verschiedene Versionen Ihrer KIs sollen gegeneinander antreten können.
- Nur Bibliotheken verwenden, die unter Windows lauffähig sind.

Bei Verwendung von Random immer Seed setzen.

In [ ]:
#!pip install chess
import random
from datetime import datetime
from typing import Tuple, Union

import chess, chess.polyglot, chess.syzygy, chess.pgn
from IPython.display import clear_output, display

Zunächst definieren wir einen Enum, um durch den folgenden Programmcode hinweg eine bessere Lesbarkeit zu gewährleisten.

In [ ]:
from enum import Enum

class State(Enum):
    OPENING_GAME = 0
    MIDDLE_GAME = 1
    END_GAME = 2
    FINISHED = 3

## Eröffnungsspiel

Für das Eröffnungsspiel wird das [Baron Polyglot Opening Book](https://www.chessprogramming.net/new-version-of-the-baron-v3-43-plus-the-barons-polyglot-opening-book/) von Richard Pijl verwendet. Hierbei handelt es sich um eine Sammlung von Eröffnungszügen im sog. 'polyglot book format'.

## Das Polyglot Book Format

Das Polyglot Book Format wird von H.G. Muller auf seiner [Website](http://hgm.nubati.net/book_format.html) sehr gut beschrieben. Die dortigen Ausführungen werden im folgenden sehr komprimiert dargestellt.

Ein Polyglot Book besteht aus einer Serie von Einträgen, wobei ein Eintrag für einen Zug in einer bestimmten Board-Situation steht und jeweils 4 Attribute aufweist:

| Attribut | Beschreibung |
|---|---|
| key (64-bit unsigned int) | Der sog. 'Zobrist' Hash der exakten Board-Position. |
| move (16-bit unsigned int) | Die Angabe des Zuges (Quelle und Ziel als Zeile & Spalte) und ob eine Figur-Umwandlung stattfindet. |
| weight (16-bit unsigned int) | Die Qualität des Zuges, berechnet als $2 \cdot (gewonnene Spiele) + (verlorene Spiele)$, skaliert auf 16-bit. |
| learn (32-bit unsigned int) | Ein Feld, welches dafür benutzt werden kann, um Erfahrungen mit dem Zug zu speichern (bspw. wie oft und mit welchem Erfolg dieser Zug gespielt wurde) |

Die Python `chess` Bibliothek bietet für die Arbeit mit Polyglot Opening Books die Klasse [`chess.polyglot.MemoryMappedReader`](https://python-chess.readthedocs.io/en/latest/polyglot.html) an. Eine Instanz dieser Klasse kann mithilfe der Funktion `chess.polyglot.open_reader("Pfad_zur_Polyglot_Bibliotheksdatei")` erzeugt werden.

In [ ]:
# Definition of the opening book reader
opening_game_reader = chess.polyglot.open_reader("opening/Baron/baron30.bin")

Diese nun erzeugte Klasse bietet die folgende Funktion an:
- `get(board)`: Gibt für ein gegebenes Board den (ersten) Eintrag mit dem höchsten `weight` zurück.  
  Die gekürzte Funktionssignatur lautet `get(board: Union[chess.Board, int]) → Union[chess.polyglot.Entry, None]`

Diese Funktion wird in `get_next_opening_game_move()` dazu verwendet den jeweils besten Zug während des Eröffnungsspiels zu ermitteln.

In [ ]:
def get_next_opening_game_move() -> chess.Move:
    '''Picks the best game entry out of all opening book entries and returns its move.'''
    return opening_game_reader.get(board).move

Die Funktion `get_next_middle_game_move()` wird verwendet, solange die gegebene Partie sich im Middlegame befindet und gibt einen zufällig ausgewählten Zug aus der Menge aller gültigen Züge zurück.

In [ ]:
def get_next_middle_game_move() -> chess.Move:
    '''Picks a random move out of all legal board moves'''
    moves = list(board.legal_moves)
    move = moves[random.randint(0, len(moves) - 1)]
    return move

Diese Funktion `get_next_end_game_move()` wird aufgerufen, solange die gegebene Partie sich im Endgame befindet und gibt einen Zug ...  
TBD @piuswalter

In [ ]:
# Definition of the end game tablebase reader
end_game_reader = chess.syzygy.open_tablebase("ending/3-4-5")

In [ ]:
def get_next_end_game_move() -> chess.Move:
    moves = []
    for move in board.legal_moves:
        board.push(move)
        wdl = end_game_reader.get_wdl(board)
        dtz = end_game_reader.get_dtz(board)
        board.pop()
        moves.append((move, wdl, dtz))
    best_wdl = max(moves, key=lambda move: move[1])[1]
    moves = [move for move in moves if move[1] == best_wdl]
    best_move = min(moves, key=lambda move: move[2])[0]
    return best_move

Diese Funktion gibt den nächsten Spielzug, sowie den neuen Spielstatus zurück.

- Falls der aktuelle Spielstatus dem des Eröffnungsspiels entspricht, wird ein neuer Spielzug aus der Polyglot-Eröffnungsbibliothek zurückgegeben.
- Wenn der Status **nicht** dem des Eröffnungsspiels entspricht, wird überprüft, ob die Endspiel-Bibliothek bereits einen Spielzug für die aktuelle Spielsituation findet.
  - Falls ja, wird der nächste Spielzug und Status aus der dazugehörigen Hilfsfunktion zurückgegeben.
- Sollte die Endspiel-Bibliothek **keinen** gültigen Zug finden, wird überprüft, ob auf dem Spielbrett noch gültige Spielzüge verfügbar sind.
    - Sind gültige Züge vorhanden, so wird die Hilfsfunktion für das Mittelspiel aufgerufen.
- Ist dies **nicht** der Fall, so wird das Spiel beendet.

In [ ]:
def get_next_move(state: State) -> Tuple[Union[chess.Move, None], State]:
    '''Figure out the next possible move and switch between different game states if neccessary'''   
    # Abort when fifty-move rule applies
    if board.can_claim_fifty_moves():
        return None, State.FINISHED
    # If there is a move in the opening library find the best one
    elif state == State.OPENING_GAME and opening_game_reader.get(board):
        return get_next_opening_game_move(), State.OPENING_GAME
    # If the endgame library provides a move use it
    elif end_game_reader.get_wdl(board) != None:
        return get_next_end_game_move(), State.END_GAME
    # If neither the opening nor the endgame library has any moves available choose randomly
    elif list(board.legal_moves):
        return get_next_middle_game_move(), State.MIDDLE_GAME
    # If there are no moves at all return none and end game
    else:
        return None, State.FINISHED

Hier wird ein neues Spiel initialisiert und ein fester Seed definiert. Der statische Seed dient dazu, sämtliche Zufallsfunktionen reproduzierbar zu machen. Der Spielzustand wird initial auf `OPENING_GAME` gesetzt.

Im Anschluss wird die `get_next_move()` Funktion in Dauerschleife so lange ausgeführt, bis sie keinen Spielzug mehr zurückliefert.

In [ ]:
# Create a new board
board = chess.Board()

# Set a seed, so randomization is reproducable
random.seed(1)

# Start with opening game
state = State.OPENING_GAME

while True:
    # Get next move
    next_move, next_state = get_next_move(state)

    # Push next move or finish game
    if next_move:
        board.push(next_move)
    else:
        print(f"Finished with: {state} -> {next_state}")
        break

    # Print board
    clear_output(wait=True)
    display(board)

    # Log state transitions
    if next_state != state:
        print(f"\n Transition from {state} to {next_state} \n")
        input("Press enter to continue")
        # Close ployglot opening book to save memory if opening game has finished 
        if state == State.OPENING_GAME:
            opening_game_reader.close()
        state = next_state

# Close endgame tablebase reader
end_game_reader.close()

Optional kann die Partie abschließend im PGN-Format gespeichert werden.

In [ ]:
# Save game to src/games/YYYY-MM-DD_HH-MM-SS.pgn (based on current time)

game = chess.pgn.Game.from_board(board)
game.headers["Event"] = "Chess-AI game"
game.headers["Date"] = datetime.now().strftime("%d.%m.%Y")
filename = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
with open(f"games/{filename}.pgn", 'w') as gamefile:
    gamefile.write(str(game))
